In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import csv 
import requests
import glob
from pandas import DataFrame
from pandas import concat
from util import read_multiple_csv, select_relevant_rows, series_to_supervised, select_attributes, read_movement_data

In [ ]:
url_regional_data = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-PLACEHOLDER.csv'
path = r'/Users/marcomazzoli/Documents/Projects/COVID-19/dati-regioni'

frame = read_multiple_csv(path)


In [ ]:
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'

region_focus_data = select_relevant_rows(frame, attribute_focus, region_focus)
#region_focus_data.head()

In [ ]:
frame_interesting_columns = select_attributes(region_focus_data, ['data', 'ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati'])
data = series_to_supervised(frame_interesting_columns, n_in=20, n_out=3)
#print(data)

In [ ]:
frame_interesting_columns = pd.DataFrame(frame_interesting_columns)

In [ ]:
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(r'%Y-%m-%d')


In [ ]:
frame_interesting_columns

In [ ]:
"""
DataSet crea istanze della serie temporale, con variabili parametriche, ogni istanza ha i valori considerati (da file DataSet) per un valore parametrico di giorni antecedenti.
Provincia (regione) (si puo’ scendere per comune nel caso sia disponibile, ma se i valori sono inferiori a 30 difficile fare training).
Età in fasce (se disponibile, lavorare per fasce, numero per fascia considerata, almeno 4-5). (Assente da protezione civile.)
Genere (in due variabili, numero uomini, numero donne, se presente da dati IRST). (Assente da protezione civile.)
Numero di tamponi effettuati (tasso ogni 100.000 residenti).
Numero di tamponi effettuati su nuovi soggetti (tasso ogni 100.000 residenti).
Numero di tamponi positivi su nuovi soggetti (tasso ogni 100.000 residenti).
Numero di tamponi positivi su totale tamponi effettuati (tasso ogni 100.000 residenti).
Popolazione residente.
Densità di popolazione.
Dati mobilita’ di Google.

Output
Numero di pazienti ricoverati (tasso ogni 100.000 residenti).
Numero di pazienti ricoverati in terapia intensiva (tasso ogni 100.000 residenti).
Numero di decessi (tasso ogni 100.000 residenti).
"""

In [ ]:
path = '/Users/marcomazzoli/Documents/Scuola/Region_Mobility_Report_CSVs'
df = read_movement_data(path, 'IT_Region_Mobility_Report', region='Emilia-Romagna')

In [ ]:
df

In [ ]:
df = df[['date',
            'retail_and_recreation_percent_change_from_baseline',
            'grocery_and_pharmacy_percent_change_from_baseline',
            'parks_percent_change_from_baseline',
            'transit_stations_percent_change_from_baseline',
            'workplaces_percent_change_from_baseline',
            'residential_percent_change_from_baseline']]

In [ ]:
df

In [ ]:
frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)

In [ ]:
merged = pd.merge(frame_interesting_columns,df,how='outer',on='date')


In [ ]:
frame_interesting_columns.set_index('date',inplace=True)
df.set_index('date',inplace=True)

In [ ]:
merged = pd.merge(frame_interesting_columns,df,on='date')


In [ ]:
merged